In [1]:
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [2]:
import tensorflow as tf
from modules.models import Model_3
from modules.utils import get_dataset
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

2023-02-28 20:42:13.123012: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
ROOT_PATH = "/home/jovyan/ChestXray-14"
INPUT_PATH = f"{ROOT_PATH}/dataset/ChestXray NIH"
LABELS = ['No Finding']

In [4]:
EXPERIMENT_NAME = "cross_entropy_model_3"

In [5]:
class Dataset:
    INPUT_PATH = INPUT_PATH
    
    def get_train(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/binary_under_sampling/train/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

    def get_test(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/binary_under_sampling/test/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

## Model Training

In [6]:
import os
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/experiments/BinaryModelWithUnderSampling'

In [7]:
# Learning rate
def lr_schedule(epoch, learning_rate):
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [8]:
def get_callbacks(NAME, weight_option, fold_num=None):
    model_checkpoint_callback = ModelCheckpoint(f'results/models/{EXPERIMENT_NAME}/{NAME}_{weight_option}.h5', monitor='val_loss', mode='min', save_best_only=True)
    early_stop_callback = EarlyStopping(monitor='val_loss', mode="min", patience=20, verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.5, patience=3, verbose=1)
    lr_logging_callback = LearningRateScheduler(lr_schedule)
    
    return model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback

In [9]:
# Constant variables
NAME = "EfficientNetB0"
EPOCHS = 100

In [10]:
weight_option = None # use `imagenet` or `None` only

In [11]:
# Callbacks
model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback = get_callbacks(NAME, weight_option)

# Path for CSV
path = os.path.join(CURRENT_PATH, "results", "history", f"training_with_{EXPERIMENT_NAME}", f"{NAME}_{weight_option}")
os.makedirs(path, exist_ok=True)

# CSV Logger
csv_logger = CSVLogger(os.path.join(path, f"history.csv"))

# Dataset
train_dataset, test_dataset = Dataset().get_train(), Dataset().get_test()

# Modeling
transfer_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False, 
    weights=weight_option,
    input_shape=(224, 224, 3),
    pooling=None
)

model = Model_3(
    transfer_model,
)
model.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model = model.get_model()
model._name = "BinaryModelCrossEntropyOnSamplingDataset"
model.summary()

2023-02-28 20:42:15.392611: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 20:42:17.039823: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31709 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:ca:00.0, compute capability: 8.0


Model: "BinaryModelCrossEntropyOnSamplingDataset"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 512)               32113152  
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_3 (Dense)          

In [12]:
# Visualize
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    verbose=1, # Show Progress Bar while Traning
    callbacks=[model_checkpoint_callback, csv_logger, early_stop_callback, reduce_lr_callback, lr_logging_callback]
)

Epoch 1/100


2023-02-28 20:42:28.746306: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-28 20:42:29.733211: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-28 20:42:29.734004: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-28 20:42:29.734035: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-28 20:42:29.734852: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-28 20:42:29.734931: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-02-28 20:42:34.066279: I tensorflow/stream_executor/cuda/c

3060/3060 [==============================] - 654s 208ms/step - loss: 0.4460 - auc: 0.5035 - val_loss: 0.4313 - val_auc: 0.5055 - lr: 0.0010
Epoch 2/100
3060/3060 [==============================] - 622s 203ms/step - loss: 0.4326 - auc: 0.5067 - val_loss: 0.4334 - val_auc: 0.4849 - lr: 0.0010
Epoch 3/100
3060/3060 [==============================] - 623s 204ms/step - loss: 0.4320 - auc: 0.5028 - val_loss: 0.4318 - val_auc: 0.5029 - lr: 0.0010
Epoch 4/100
3060/3060 [==============================] - ETA: 0s - loss: 0.4324 - auc: 0.4954
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
3060/3060 [==============================] - 624s 204ms/step - loss: 0.4324 - auc: 0.4954 - val_loss: 0.4340 - val_auc: 0.4844 - lr: 5.0000e-04
Epoch 5/100
3060/3060 [==============================] - 623s 204ms/step - loss: 0.4273 - auc: 0.5544 - val_loss: 0.4165 - val_auc: 0.6451 - lr: 5.0000e-04
Epoch 6/100
3060/3060 [==============================] - 622s 203ms/step - loss: 0.420